# Similarity Functions

This notebook describes about the similarity functions that can be used to measure the similarity between two sets.

Firstly we import the shingling functions and other helpful functions.

In [ ]:
from src.shingle import *
from math import ceil, floor
import numpy as np

We will then count how frequent a shingle is in the document. For this I have calculated the frequencies in the document called `data/portuguese/two_ends.txt`. Here we are using portuguese corpus.

Then we create a dictionary called `frequencies` which goes like from the word to its frequency.

In [ ]:
# Initialize counts
frequencies = {}
text = open("data/portuguese/two_ends.txt", "r+")
for line in text:
    word = line.strip().split(' ')
    frequencies[word[0]] = float(word[1])   

## TF - IDF

TF-IDF (Term-frequency and Inverse Document Frequency) measures similarity using this:
<img src="utils/tfidf.png" alt="tfidf" width="400px"/>

Firstly, we define `tf` using this, which is just the frequency counts in the intersection.

In [ ]:
def tf(intersection, query):
    '''Counts term frequency'''
    tf = [query.count(word) for word in intersection]
    return np.array(tf)

Afterwards, we compute `idf`, which is inverse document frequency. Here we will make use of the dictionary that we created earlier in order to compute document frequencies.

In [ ]:
def idf(intersection, document, N):
    '''Counts inverse document frequency'''
    idf = np.array([frequencies[word] for word in intersection])
    idf = np.log10(np.divide(N + 1, idf + 0.5))
    return idf

Finally we simulate the function `tf_idf` which takes the dot product of `tf` and `idf` arrays.

In [ ]:
def tf_idf(query, document, N):
    intersection = [word for word in document if word in query] # intersection
    score = np.dot(tf(intersection, query), idf(intersection, document, N))
    return score

We can then run the similarity function in the following manner:

In [ ]:
query = two_ends("pizza", 2)
document = two_ends("pizza", 2)
tf_idf(query, document, 50000)

## BM25

The formula of BM25 is given like this:

<img src="utils/bm25.png" alt="tfidf" width="800px"/>

Here we define the `bm25_tf` in the following manner:

In [ ]:
def bm25_tf(intersection, query, document, k1, b, avgdl, N):
    tf_ = tf(intersection, document)
    numerator = tf_ * (k1 + 1.0)
    denominator = tf_ + k1 * (1.0 - b + b * (len(query) / avgdl))
    bm25_tf = np.divide(numerator, denominator)
    return bm25_tf

Finally we will take the dot product of `bm25_tf` and `idf` to get this:

In [ ]:
def bm25(query, document, k1 = 1.2, b = 0.75, avgdl = 8.3, N = 50000):
    intersection = [word for word in document if word in query] # intersection
    score = np.dot(bm25_tf(intersection, query, document, k1, b, avgdl, N), idf(intersection, document, N))
    return score

We can run the function in the following manner:

In [ ]:
query = two_ends("pizza", 2)
document = two_ends("pizza", 2)
bm25(query, document)

## Dirichlet

The formula of Dirichlet is given like this:

$$
Dir(q, d) = \sum_{t \in q, d} c(t, q) \log \Bigg(1 + \frac{c(t, d)}{\mu \cdot p(t | C)}\Bigg) + |q| \log \frac{\mu}{\mu + |d|}
$$

Firstly, we will compute the sum dependent function here in the form of `smooth`.

In [ ]:
shingles = 470751
def smooth(intersection, document, mu):
    smooth = []
    for word in intersection:
        prob = 1.0 + np.divide(document.count(word), mu * frequencies[word] / shingles)
        smooth.append(np.log10(prob))
    smooth = np.array(smooth)
    return smooth

We will add the sum independent function to `smooth` and take the dot product to `tf`.

In [ ]:
def dirichlet(query, document, mu = 100.0):
    intersection = [word for word in document if word in query] # intersection
    add = len(query) * np.log10(np.divide(mu, mu + len(document)))
    score = np.dot(tf(intersection, query), smooth(intersection , document, mu)) + add
    return score

We can this function in following manner:

In [ ]:
query = two_ends("pizzzza", 2)
document = two_ends("pizzza", 2)
print(dirichlet(query, document))